![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Science/DigitalDataCollection/digital-data-collection.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Digital Data Collection

We can collect data in this notebook using a micorcontroller device like a [micro:bit](https://www.microbit.org), [Arduino](https://www.arduino.cc), [Raspberry Pi Pico](https://www.raspberrypi.org/products/raspberry-pi-pico), or anything that can connect to your computer and send [serial data](https://en.wikipedia.org/wiki/Serial_communication).

Most of these devices have [onboard sensors or inputs](https://microbit.org/get-started/first-steps/sensors), and you can also connect [external sensors](https://pinouts.net/arduino-sensor-list). For this experiment we will be using the temperature sensor on a micro:bit.

## Setting Up

If you are not familiar with programming a micro:bit, follow the [first steps](https://microbit.org/get-started/first-steps/set-up) instructions.

We want to program the micro:bit to send time-stamped temperature readings using the following MakeCode blocks:

[![temperature code](images/makecode-temperature.jpg)](https://makecode.microbit.org/_Kve3bF5ibEzz)

If you prefer to program the micro:bit in Python, use the following code:

```python
def write_values():
    serial.write_string(str(input.running_time()))
    serial.write_string(',')
    serial.write_string(str(input.temperature()))
    serial.write_line('')
    basic.pause(500)
basic.forever(write_values)
```

## Collecting Data

Once your device is sending data over the serial port, select the html code cell below and click the `▶Run` button.

In [ ]:
%%html
<body>
    <div class="buttons">
        <button id="connect" class="button">Connect</button>
        <button id="disconnect" disabled class="button">Disconnect</button>
        <button id="clearData" class="button">Clear</button>
        <button id="saveData" class="button">Save</button>
        <button id="generateDemoData" class="button">Generate Demo Data</button>
    </div>
    <p id="dataArea"></p>
</body>
<script type="text/javascript">
Jupyter.notebook.scroll_cell_percent(Jupyter.notebook.get_selected_index()+1, 50, 1000);
if (!('serial' in navigator)) {document.getElementById("dataArea").innerText="This only works in Chrome or Edge browsers";}
var decoder = new TextDecoder();
var connect = document.querySelector("#connect");
var disconnect = document.querySelector("#disconnect");
var port;
var reader;
var dataPoints;
async function readLoop() {
    while (true) {
        var {value, done} = await reader.read();
        if (value) {
            document.getElementById("dataArea").innerText += decoder.decode(value);
            dataPoints += decoder.decode(value)
        }
        if (done) {
            reader.releaseLock();
            break;
        }
    }
}
connect.addEventListener("click", async () => {
    port = await navigator.serial.requestPort();
    await port.open({baudRate: 115200}); // this number may need to be changed for devices other than micro:bit
    reader = port.readable.getReader();
    connect.disabled = true;
    disconnect.disabled = false;
    readLoop();
});
disconnect.addEventListener("click", async () => {
    await reader.cancel();
    await port.close();
    disconnect.disabled = true;
    connect.disabled = false;
});
clearData.addEventListener("click", async () => {
    document.getElementById("dataArea").innerText = "";
});
saveData.addEventListener("click", async () => {
    var dataFromDevice = "%%writefile data.csv" + "\n" + document.getElementById("dataArea").innerText;
    Jupyter.notebook.select_next();
    Jupyter.notebook.select_next().get_selected_cell().set_text(dataFromDevice);
    Jupyter.notebook.execute_cell();
    Jupyter.notebook.select_next();
    Jupyter.notebook.execute_cell();
    Jupyter.notebook.scroll_to_cell(Jupyter.notebook.get_selected_index(), 1000);
});
generateDemoData.addEventListener("click", async () => {
    document.getElementById("dataArea").innerText = "10756,16\n11262,17\n11769,17\n12275,17\n12782,17\n13289,16\n13796,16";
});
</script>

The `Connect` button will prompt you connect your `USB Serial Device`. Once it is connected, you should start to see data streaming in.

You can use the `Clear` button to restart the data collection, and the `Save` button when you have finished.

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv', header=None)
df

You now have your data in a [DataFrame](https://www.geeksforgeeks.org/python-pandas-dataframe) called `df`. If there were any errors, you can go back to the cell that starts with `%%writefile data.csv` and change values, then run that cell and the `import pandas as pd` cell again.

### Column Labels

Use the following code cell to label the columns in your DataFrame.

In [ ]:
df.columns = ['Time', 'Temperature']
df

## Graphing Data

We can now graph the data as a `scatter`, `line`, `bar`, or [many other types](https://plotly.com/python/plotly-express).

The lines `x = 'Time'` and `y = 'Temperature'` should match your column titles above.

Try changing the last line in the following code cell to `px.line(df, x=x, y=y, title=y+' versus '+x)`

In [ ]:
import plotly.express as px
x = 'Time'
y = 'Temperature'
px.scatter(df, x=x, y=y, title=y+' versus '+x)

Use the following code to generate a graph with a linear trendline. For a non-linear trenline change `'ols'` to `'lowess'`.

In [ ]:
px.scatter(df, x=x, y=y, title=y+' versus '+x, trendline='ols')

We can also calculate the slope and y-intercept of a linear trendline.

You can change `, 1)` to `, 2)` for a quadratic fit.

In [ ]:
import numpy as np
fit = np.polyfit(df[x], df[y], 1)
print('Slope =', fit[0])
print('y-intercept =', fit[1])

# Conclusion

This notebook demonstrated collecting data in Jupyter notebook using an internal sensor on a micro:bit, a similar process would be used for other internal or external sensors.

You can also send serial data using Arduino's [Serial.write()](https://www.arduino.cc/reference/en/language/functions/communication/serial/write) or similar functions in other microcontrollers or robots.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)